In [1]:
import ifcopenshell
from ifctester import reporter
from ifctester import ids

No stream support: No module named 'lark'


# Lage IDS TEK

[IDS docutmentation](https://github.com/buildingSMART/IDS/blob/master/Documentation) in buildingSMART github repo. 

[TEK og BIM miro tavle](https://miro.com/app/board/uXjVMRfSzQc=/)


## Lage IDS
Start by folling the [metadata guidelines found here.](https://github.com/buildingSMART/IDS/blob/master/Documentation/ids-metadata.md)

## §12-8. Entré og garderobe
Vi lager en prototype på IDS for å sjekke opp mot [§12-8. Entré og garderobe](https://www.dibk.no/regelverk/byggteknisk-forskrift-tek17/12/ii/12-8). 

Ambisjonen er å kunne lage en IDS som sikrer at modeller kan valideres mot IDSen for å ha tilsterekkelig informasjon for å kunne tilfredstille de preaksepterte ytelsene. 

Med IDS kan vi lage maskinlesbare krav til at modellene lar seg sjekke, i tillegg til å sjekke på faktiske verdier i modellen. Dette gjør at de fleste kravene kan sjekkes automatisk ved f.eks. eksport fra tegneverktøy eller opplastning til filplassering. Noen krav vil måtte sjekkes geometrisk, som da må gjøres i programvare som er egnet for slike sjekker. Det vil typisk være sjekk på om to volumer overlapper, slik som må gjøres for å sjekke om snuarealet ikke overlapper med slagarealet til den døren som skal betjenes, slik som bilde under viser. 

![§ 12-8 Figur 1: Snuarealet må ikke overlappe slagarealet til den døren som skal betjenes.](https://www.dibk.no/regelverk/byggteknisk-forskrift-tek17/12/ii/12-8/_/image/d0503953-c0e9-4142-94c5-993fd54c3df1:a7f679aad4851a957332ed68509ec1d329f1cc19/width-768/-12-8-figur-1-snuarealet-ma-ikke-overlappe-slagarealet-til-den-doren-som-skal-betjenes..png)

I dette tilfellet kan IDS for §12-8. Entré og garderobe likevel støtte prosessen med at man krever at objektene, eks snuareal og dør, er tilstede og har egenskapene som skal til for å kunne valideres i slik programvare. 

In [2]:
# create new IDS 
# ref https://github.com/buildingSMART/IDS/blob/master/Documentation/ids-metadata.md 

TEK17_1208_ids = ids.Ids(
   title="§12-8. Entré og garderobe",
   author="sigve@mok-see.com",
   copyright="[mok-see] AS",
   version="0.01",
   purpose= "Denne IDS filen er laget som en demonstrator for TEK og BIM for validering av §12-8. Entré og garderobe",
   description= "Som en del av digitalisering av TEK er denne IDS (Information Delivery Specification) filen laget med bakgrunn i å kunne validere at BIM modeller tilfredstiller §12-8. Entré og garderobe",
   milestone="MMI 500: As-Built")

### (1) Entré eller inngang i en tilgjengelig boenhet skal ha

#### a) fri passasje på minimum 0,9 m utenfor møbleringssonen 

Fri passasje skal i dette tilfellet sjekkes på "Tilgjengelig boenhet" og i området betegnet Entré eller inngang. I denne situasjonen skal fri passasje være 0.9 m i bredde. 

Vi lager derfor følgende krav: 
 1) Bygningstypen skal være "Tilgjengelig bolig" som definert av DiBK
 2) Det skal være angit et område for Entré/inngang
 3) Inne i Entré/inngang skal det være definert et område for Fri passasje som er over 0.9 m i bredde.

*Merk: Bredden på fri passasje varierer avhengig av hvor det er i en bolig, f.eks. vil det ligge mellom 1.2 m og 1.5 m i korridor/kommunikasjonsvei.*






##### 1) Spesifisere bygningstype

In [3]:
""" 

1) Spesifikasjon for sjekk av bygningstype

"""

bygningstype_spec = ids.Specification(
    name="Bygningstype",
    description= "Bygningstype må angi om det er en Tilgjengelig boenhet eller ikke",
    instructions="IfcBuilding skal brukes til å angi bygningstype iht. TEK17 definisjoner",
    minOccurs=1,
    ifcVersion="IFC4"
)
bygningstype_spec.applicability.append(
    ids.Entity(
        name = "IfcBuilding"
    )
)

bygningstype_spec.requirements.append(
    ids.Attribute(
        name = "LongName",
        value = ids.Restriction(
            options={"enumeration":[
                "Tilgjengelig Bolig",
                "Tilgjengelig Boenhet"
                ]
            }
        )
    )
)
bygningstype_spec.requirements.append(
    ids.Property(
        propertySet="Pset_BuildingCommon",
        name ="OccupancyType",
        value= "Tilgjengelig boenhet",
        datatype = "IfcLabel",
        uri="https://www.dibk.no/regelverk/byggteknisk-forskrift-tek17/12/i/innledning2#:~:text=personer%20i%20rullestol.-,Tilgjengelig%20boenhet,-Boenhet%20der%3A",
        instructions="For å sjelle opp mot §12-8. Entré og garderobe i TEK17 må OccupancyType være deklarert som Tilgjengelig boenhet"
    )
)
TEK17_1208_ids.specifications.append(bygningstype_spec)


##### 2) Spesifisere krav til område for Entré / Inngang / Inngagsplan 

[Tilgjengelig boenhet](https://www.dibk.no/regelverk/byggteknisk-forskrift-tek17/12/i/innledning2#:~:text=personer%20i%20rullestol.-,Tilgjengelig%20boenhet,-Boenhet%20der%3A)
Boenhet der:

Alle hovedfunksjoner finnes på inngangsplanet, og
Inngangsplanet er tilgjengelig for personer med nedsatt bevegelighet.

Det må følgelig defineres et romlig område av typen "Inngangsplan", iht. 

[Inngangsplan](https://www.dibk.no/regelverk/byggteknisk-forskrift-tek17/12/i/innledning2#:~:text=med%20nedsatt%20bevegelighet.-,Inngangsplan,-Bygningens%20inngangsplan%20er)
* Bygningens inngangsplan er det planet der bygningens hovedinngang er plassert. En bygning kan ha flere hovedinnganger.
* Boenhetens inngangsplan er det planet hvor boenhetens inngangsdør er plassert.
* Øvrige plan er de plan som har en høydeforskjell på mer enn 1,2 meter fra inngangsplanet. Er høydeforskjellen inntil 1,2 meter er det fortsatt inngangsplan.


In [4]:
""" 

2)a Spesifikasjon for sjekk av Inngangsplan

"""

inngangsplan_spec = ids.Specification(
    name="Inngangsplan",
    description= "Inngangsplan må være definert og romme alle hovedfunksjoner, samt være der bygningsens hovedinngang er plassert",
    instructions="Det skal eksistere minimum ett objekt til å definere Inngangsplanet i modellen iht TEK17",
    minOccurs=1,
    ifcVersion="IFC4"
)

# Vurdere å legge inn alternativ oppfyllelse, ved bruk av Rom med classe 511 Adkomst. 
inngangsplan_spec.applicability.append(
    ids.Entity(
        name = "IfcSpatialZone"
    )
)

inngangsplan_spec.applicability.append(
    ids.Attribute(
        name = "Name", 
        value= ids.Restriction(
            options={"enumeration":[
                "Inngangsplan",
                "Entré",
                "Inngang",
                "511 Adkomstrom"
                ]
            }
        )
    )
)
# Må ha plassering for å kunne sjekke 1.2 m kravet ift. andre plan. 
inngangsplan_spec.requirements.append(
    ids.Attribute(
        name = "ObjectPlacement",
        instructions="Inngangsplanet må ha en definert plassering, slik at man kan stadfeste høyde til andre plan og objekter i modell"
    )
)

# Kreve at de oppgir type iht. TEK17 ordbok / typekatalog
inngangsplan_spec.requirements.append(
    ids.Attribute(
        name = "ObjectType",
        value= ids.Restriction(
            options={"enumeration":[
                "Inngangsplan",
                "Entré",
                "Inngang",
                "511 Adkomstrom"
                ]
            }
        ),
        instructions="Inngangsplanet må ha definert typebetegnelse iht. TEK 17"
    )
)
TEK17_1208_ids.specifications.append(inngangsplan_spec)

In [5]:
""" 

2)b Spesifikasjon for sjekk av dør inne i inngangsplan

"""
# Må være inne i IfcSpatialStructure
# Operasjon 
# Slagareal 
# Slagretning 

dor_inngangsplan_spec = ids.Specification(
    name="Inngangsplan - dører",
    description= "Inngangsplanet for Tilgjengelig boenhet skal ha dører med terskelfrihet og trinnfrihet",
    instructions="Det eksisterer dørobjekter som har oppgitt slagareal, slagretning og deklarering av tilgjengelighet for funksjonshemmede",
    minOccurs=1,
    ifcVersion="IFC4"
)
""" APPLICABILITY """
# Vurdere å legge inn alternativ oppfyllelse
dor_inngangsplan_spec.applicability.append(
    ids.Entity(
        name = "IfcDoor"
        )
    )


""" REQUIREMENTS """
# Objekt som selv er innenfor et annet IfcSpatialObject (av egendefinert type)
## NB: Sjekker bare dører som er innenfor "Inngangsplan"
dor_inngangsplan_spec.requirements.append(
    ids.PartOf(
        entity="IfcSpatialZone",
        relation="IFCRELCONTAINEDINSPATIALSTRUCTURE"
    )
)

dor_inngangsplan_spec.requirements.append(
    ids.Attribute(
        name = "ObjectPlacement",
        instructions="Dørobjektet må ha en definert plassering"
    )
)

dor_inngangsplan_spec.requirements.append(
    ids.Attribute(
        name = "Representation",
        instructions="Dørobjektet må ha en definert og representativ geometri"
    )
)

# Kreve at de oppgir type iht. TEK17 ordbok / typekatalog
dor_inngangsplan_spec.requirements.append(
    ids.Property(
        propertySet="Pset_DoorCommon",
        name = "HandicapAccessible",
        value = "TRUE",
        datatype = "IfcBoolean",
        minOccurs=1,
        instructions="Døren må være tilgjengelig for funksjonshemmede"
    )
)
# Kreve at de oppgir type iht. TEK17 ordbok / typekatalog
dor_inngangsplan_spec.requirements.append(
    ids.Attribute(
        name = "OverallWidth",
        value = ids.Restriction(
            options={ 
                "minInclusive": 0.9
            }
        ),
        instructions="Dørbredden må være definert og minimum 0.9 m"
    )
)
TEK17_1208_ids.specifications.append(dor_inngangsplan_spec)

#### 3) Spesifikasjon av Fri passasje inne i Inngangsplan

Inngangsplanet skal sikre fri passasje på minimum 0.9 m. 




In [6]:
""" 

3) Spesifikasjon for sjekk av Fri passasje

"""

fri_passasje_spec = ids.Specification(
    name="Fri passasje",
    description= "Inngangsplanet for Tilgjengelig boenhet skal ha fri passasje på minimum 0.9 m",
    instructions="Det skal eksistere ett objekt i inngangsplanet som angir fri passasje på minimum 0.9 m",
    minOccurs=1,
    ifcVersion="IFC4"
)
""" APPLICABILITY """
# Vurdere å legge inn alternativ oppfyllelse, ved bruk av Rom med classe 511 Adkomst. 
fri_passasje_spec.applicability.append(
    ids.Entity(
        name = "IfcSpatialZone"
        )
    )

fri_passasje_spec.applicability.append(
    ids.Attribute(
        name = "Name", 
        value= "Fri passasje"
    )
)
""" REQUIREMENTS """
# Objekt som selv er innenfor et annet IfcSpatialObject (av egendefinert type)
fri_passasje_spec.requirements.append(
    ids.PartOf(
        entity="IfcSpatialZone",
        relation="IFCRELCONTAINEDINSPATIALSTRUCTURE"
    )
)



fri_passasje_spec.requirements.append(
    ids.Attribute(
        name = "ObjectPlacement",
        instructions="Fri passasje objektet må ha en definert plassering"
    )
)

fri_passasje_spec.requirements.append(
    ids.Attribute(
        name = "Representation",
        instructions="Fri passasje objektet må ha en definert og representativ geometri"
    )
)

# Kreve at de oppgir type iht. TEK17 ordbok / typekatalog
fri_passasje_spec.requirements.append(
    ids.Property(
        propertySet="NODIBK_TEK17",
        name = "FreePassage",
        value = ids.Restriction(
            options={ 
                "minInclusive": 0.9
            }
        ),
        datatype = "IfcPositiveLengthMeasure",
        minOccurs=1,
        instructions="Fri passasje objektet må ha et en definert bredde på mer enn 0.9 m"
    )
)
# Kreve at de oppgir type iht. TEK17 ordbok / typekatalog
fri_passasje_spec.requirements.append(
    ids.Attribute(
        name = "ObjectType",
        value= "Fri passasje",
        instructions="Fri passasje må ha definert typebetegnelse iht. TEK 17"
    )
)
TEK17_1208_ids.specifications.append(fri_passasje_spec)

#### b)  plass til snuareal for rullestol utenfor dørens slagradius.

Det må være etablert et objekt for å angi korrekt snuareal som kan sjekkes at ikke har geometrisk konflikt med dørenes slagradius i inngangsplanet. 


In [7]:
""" 

3) Spesifikasjon for sjekk av snuareal

"""

snuareal_spec = ids.Specification(
    name="Snuareal",
    description= "Inngangsplanet for Tilgjengelig boenhet skal ha plass til snuareal for rullestol utenfor dørens slagradius",
    instructions="Det skal eksistere en snusirkel eller snurektangel i inngangsplanet som har geometri som kan sjekkes at ikke går i konflikt med dørens slagradius",
    minOccurs=1,
    ifcVersion="IFC4"
)
""" APPLICABILITY """
# Vurdere å legge inn alternativ oppfyllelse, ved bruk av Rom med classe 511 Adkomst. 
snuareal_spec.applicability.append(
    ids.Entity(
        name = "IfcSpatialZone"
        )
    )

snuareal_spec.applicability.append(
    ids.Attribute(
        name = "Name", 
        value= ids.Restriction(
            options={"enumeration":[
                "Snusirkel",
                "Snurektangel"
                ]
            }
        ),
    )
)

""" REQUIREMENTS """

# Objekt som selv er innenfor et annet IfcSpatialObject (av egendefinert type)
snuareal_spec.requirements.append(
    ids.PartOf(
        entity="IfcSpatialZone",
        relation="IFCRELCONTAINEDINSPATIALSTRUCTURE"
    )
)

snuareal_spec.requirements.append(
    ids.Attribute(
        name = "ObjectPlacement",
        instructions="Snuarealobjektet må ha en definert plassering"
    )
)

snuareal_spec.requirements.append(
    ids.Attribute(
        name = "Representation",
        instructions="Snuarealobjektet må ha en definert og representativ geometri"
    )
)

# Kreve at de oppgir type iht. TEK17 ordbok / typekatalog
snuareal_spec.requirements.append(
    ids.Property(
        propertySet="NODIBK_TEK17",
        name = "TurnDiameter",
        value = ids.Restriction(
            options={ 
                "minInclusive": 1.5
            }
        ),
        datatype = "IfcPositiveLengthMeasure",
        minOccurs=1,
        instructions="Snusirkel må ha et en definert diameter på mer enn 1.5 m"
    )
)
# Kreve at de oppgir type iht. TEK17 ordbok / typekatalog
snuareal_spec.requirements.append(
    ids.Property(
        propertySet="NODIBK_TEK17",
        name = "TurnWidth",
        value = ids.Restriction(
            options={ 
                "minInclusive": 1.3
            }
        ),
        datatype = "IfcPositiveLengthMeasure",
        minOccurs=1,
        instructions="Snurektangel må ha et en definert bredde på mer enn 1.3 m"
    )
)
# Kreve at de oppgir type iht. TEK17 ordbok / typekatalog
snuareal_spec.requirements.append(
    ids.Property(
        propertySet="NODIBK_TEK17",
        name = "TurnLength",
        value = ids.Restriction(
            options={ 
                "minInclusive": 1.8
            }
        ),
        datatype = "IfcPositiveLengthMeasure",
        minOccurs=1,
        instructions="Snurektangel må ha et en definert lengde på mer enn 1.8 m"
    )
)
# Kreve at de oppgir type iht. TEK17 ordbok / typekatalog
snuareal_spec.requirements.append(
    ids.Attribute(
        name = "ObjectType",
        value= ids.Restriction(
            options={"enumeration":[
                "Snusirkel",
                "Snurektangel"
                ]
            }
        ),
        instructions="Snuareal må ha definert typebetegnelse iht. TEK 17"
    )
)
TEK17_1208_ids.specifications.append(snuareal_spec)

In [8]:
TEK17_1208_ids.to_xml("../ids/§12_08. Entré og garderobe.ids")

True

In [10]:
file = ifcopenshell.open("../ifc/tek-bim_1208.ifc")

doors = file.by_type("IfcDoor")
doors 

[#1616=IfcDoor('2u3eJAuNr1MgwxhjFsnlSs',$,'Door',$,$,#2964,#1648,$,2.14499998092651,1.00999999046326,$,$,$),
 #1720=IfcDoor('3bmqBL1uvAqOOWoYKITNnz',$,'Door',$,$,#3056,#1751,$,2.14499998092651,1.00999999046326,$,$,$),
 #1795=IfcDoor('3$Y8p_wmr6qObTDwo2yoXx',$,'Door',$,$,#1884,#1826,$,$,$,$,$,$)]

## §12-10 Bod og oppbevaringsplass

In [ ]:
# create new IDS 
# ref https://github.com/buildingSMART/IDS/blob/master/Documentation/ids-metadata.md 

TEK_ids = ids.Ids(
   title="TEK og BIM",
   author="sigve@mok-see.com",
   copyright="[mok-see] AS",
   version="0.01",
   purpose= "Denne IDS filen er laget som en demonstrator for TEK og BIM for validering av §12-10 Bod og oppbevaringsplass",
   description=   "Som en del av digitalisering av TEK er denne IDS (Information Delivery Specification) filen laget med bakgrunn i å kunne validere at BIM modeller tilfredstiller §12-10 Bod og oppbevaringsplass",
   milestone="MMI 500: As-Built")


### Lage spesifikasjoner

Spesifikasjonene som kreves i denne omgang er;
1. Boenheten skal ha tilstrekkelig og egnet plass for oppbevaring av klær
2. Boenheten skal ha tilstrekkelig og egnet plass for oppbevaring av mat

In [ ]:
# Oppbevaring av klær spesifikasjon
oppbevaring_av_klar_spec = ids.Specification(
    name="§12-10 (1) Plass til oppbevaring av klær",
    description= "Boenheten skal ha tilstrekkelig og egnet plass for oppbevaring av klær",
    instructions="Produksjonsunderlag (BIM) må vise avsatt plass til oppbevaring av klær ved IfcSpatialZone eller IfcFurniture objekt.",
    minOccurs=1,
    ifcVersion="IFC4")

oppbevaring_av_klar_spec.applicability.append(ids.Entity(name="IfcSpatialZone"))
#oppbevaring_av_klar_spec.applicability.append(ids.Entity(name="IfcFurniture"))
oppbevaring_av_klar_spec.applicability.append(ids.Attribute(name="ObjectType",value="STORAGE_OF_CLOTHES"))
#oppbevaring_av_klar_spec.applicability.append(ids.Attribute(name="ObjectType",value="WARDROBE"))

name = ids.Attribute(
    name = "Name",
    instructions="Name må være angitt"
)
oppbevaring_av_klar_spec.requirements.append(name)

longName = ids.Attribute(
    name = "LongName",
    instructions="LongName må være angitt"
)
oppbevaring_av_klar_spec.requirements.append(longName)
# Legge til spesifikasjon i IDS
TEK_ids.specifications.append(oppbevaring_av_klar_spec)


In [ ]:
# Oppbevaring av mat spesifikasjon
oppbevaring_av_torrmat_spec = ids.Specification(
    name="§12-10 (2) (1) Plass til oppbevaring av mat (Tørrmat)",
    description= "Boenheten skal ha tilstrekkelig og egnet plass for oppbevaring av tørrmat",
    instructions="Produksjonsunderlag (BIM) må vise avsatt plass til oppbevaring av tørrmat med riktig objekttype.",
    minOccurs=1,
    ifcVersion="IFC4")

## Virker som at det bare kan være en Entity i Applicability, men at man kan ha andre typer facetter som legges til.. 
oppbevaring_av_torrmat_spec.applicability.append(ids.Entity(name = "IfcSpatialZone"))
oppbevaring_av_torrmat_spec.applicability.append(ids.Attribute(
    name="ObjectType",value="STORAGE_DRY_FOOD"))

name = ids.Attribute(
    name = "Name",
    instructions="Name må være angitt"
)
oppbevaring_av_torrmat_spec.requirements.append(name)

longName = ids.Attribute(
    name = "LongName",
    instructions="LongName må være angitt"
)
oppbevaring_av_torrmat_spec.requirements.append(longName)

# Legge til spesifikasjon i IDS
TEK_ids.specifications.append(oppbevaring_av_torrmat_spec)

In [ ]:
# Oppbevaring av mat spesifikasjon
oppbevaring_av_kjolevarer_spec = ids.Specification(
    name="§12-10 (2) (2) Plass til oppbevaring av mat (Kjølevarer)",
    description= "Boenheten skal ha tilstrekkelig og egnet plass for oppbevaring av kjølevarer",
    instructions="Produksjonsunderlag (BIM) må vise avsatt plass til oppbevaring av kjølevarer med riktig object type.",
    minOccurs=1,
    ifcVersion="IFC4")

oppbevaring_av_kjolevarer_spec.applicability.append(ids.Entity(name = "IfcElectricAppliance"))
#oppbevaring_av_kjolevarer_spec.applicability.append(ids.Attribute(
#    name="ObjectType",value="STORAGE_REFRIGERATED_GOODS"))
#oppbevaring_av_kjolevarer_spec.applicability.append(ids.Attribute(name="ObjectType",value="REFRIDGERATOR"))
oppbevaring_av_kjolevarer_spec.applicability.append(ids.Attribute(name="PredefinedType",value="FRIDGE_FREEZER"))

name = ids.Attribute(
    name = "Name"
)
oppbevaring_av_kjolevarer_spec.requirements.append(name)

longName = ids.Attribute(
    name = "LongName"
)
oppbevaring_av_kjolevarer_spec.requirements.append(longName)

# Legge til spesifikasjon i IDS
TEK_ids.specifications.append(oppbevaring_av_kjolevarer_spec)

In [ ]:
# Skriv til fil
TEK_ids.to_xml("../ids/§12_10_TEK_IfcFuniture_and_IfcElectricAppliance_Error_LongName.ids")

True

In [ ]:
file = ifcopenshell.open("../ifc/tek-bim.ifc")
tekids = ids.open("../ids/§12_10_TEK_IfcFuniture_and_IfcElectricAppliance_Error_LongName.ids")
tekids.validate(file)

In [ ]:
reporter.Console(tekids).report()

TEK og BIM
[FAIL] (0/1) *§12-10 (1) Plass til oppbevaring av klær
    Applies to:
        All IfcSpatialZone data
        Data where the ObjectType is STORAGE_OF_CLOTHES
    Requirements:
        The Name shall be provided
        The LongName shall be provided
            The attribute value None is empty - #2911=IfcSpatialZone('0T9InqK9T2BvhcCuJN9byU',$,'Oppbevaring av klær',$,'STORAGE_OF_CLOTHES',#3745,#2917,$,.USERDEFINED.)
[FAIL] (0/1) *§12-10 (2) (1) Plass til oppbevaring av mat (Tørrmat)
    Applies to:
        All IfcSpatialZone data
        Data where the ObjectType is STORAGE_DRY_FOOD
    Requirements:
        The Name shall be provided
        The LongName shall be provided
            The attribute value None is empty - #2930=IfcSpatialZone('2DQ02zAnL4deegroDOcto7',$,'Oppbevaring av tørrmat',$,'STORAGE_DRY_FOOD',#3750,#2936,$,.USERDEFINED.)
[FAIL] (0/1) *§12-10 (2) (2) Plass til oppbevaring av mat (Kjølevarer)
    Applies to:
        All IfcElectricAppliance data
        Da

In [2]:
# create new IDS 
# ref https://github.com/buildingSMART/IDS/blob/master/Documentation/ids-metadata.md 

TEK_ids = ids.Ids(
   title="TEK og BIM",
   author="sigve@mok-see.com",
   copyright="[mok-see] AS",
   version="0.01",
   purpose= "Denne IDS filen er laget som en demonstrator for TEK og BIM for validering av §12-10 Bod og oppbevaringsplass",
   description=   "Som en del av digitalisering av TEK er denne IDS (Information Delivery Specification) filen laget med bakgrunn i å kunne validere at BIM modeller tilfredstiller §12-10 Bod og oppbevaringsplass",
   milestone="MMI 500: As-Built")


### Lage spesifikasjoner

Spesifikasjonene som kreves i denne omgang er;
1. Boenheten skal ha tilstrekkelig og egnet plass for oppbevaring av klær
2. Boenheten skal ha tilstrekkelig og egnet plass for oppbevaring av mat

In [3]:
# Oppbevaring av klær spesifikasjon
oppbevaring_av_klar_spec = ids.Specification(
    name="§12-10 (1) Plass til oppbevaring av klær",
    description= "Boenheten skal ha tilstrekkelig og egnet plass for oppbevaring av klær",
    instructions="Produksjonsunderlag (BIM) må vise avsatt plass til oppbevaring av klær ved IfcSpatialZone eller IfcFurniture objekt.",
    minOccurs=1,
    ifcVersion="IFC4")

oppbevaring_av_klar_spec.applicability.append(ids.Entity(name="IfcSpatialZone"))
#oppbevaring_av_klar_spec.applicability.append(ids.Entity(name="IfcFurniture"))
oppbevaring_av_klar_spec.applicability.append(ids.Attribute(name="ObjectType",value="STORAGE_OF_CLOTHES"))
#oppbevaring_av_klar_spec.applicability.append(ids.Attribute(name="ObjectType",value="WARDROBE"))

name = ids.Attribute(
    name = "Name",
    instructions="Name må være angitt"
)
oppbevaring_av_klar_spec.requirements.append(name)

longName = ids.Attribute(
    name = "LongName",
    instructions="LongName må være angitt"
)
oppbevaring_av_klar_spec.requirements.append(longName)
# Legge til spesifikasjon i IDS
TEK_ids.specifications.append(oppbevaring_av_klar_spec)


In [4]:
# Oppbevaring av mat spesifikasjon
oppbevaring_av_torrmat_spec = ids.Specification(
    name="§12-10 (2) (1) Plass til oppbevaring av mat (Tørrmat)",
    description= "Boenheten skal ha tilstrekkelig og egnet plass for oppbevaring av tørrmat",
    instructions="Produksjonsunderlag (BIM) må vise avsatt plass til oppbevaring av tørrmat med riktig objekttype.",
    minOccurs=1,
    ifcVersion="IFC4")

## Virker som at det bare kan være en Entity i Applicability, men at man kan ha andre typer facetter som legges til.. 
oppbevaring_av_torrmat_spec.applicability.append(ids.Entity(name = "IfcSpatialZone"))
oppbevaring_av_torrmat_spec.applicability.append(ids.Attribute(
    name="ObjectType",value="STORAGE_DRY_FOOD"))

name = ids.Attribute(
    name = "Name",
    instructions="Name må være angitt"
)
oppbevaring_av_torrmat_spec.requirements.append(name)

longName = ids.Attribute(
    name = "LongName",
    instructions="LongName må være angitt"
)
oppbevaring_av_torrmat_spec.requirements.append(longName)

# Legge til spesifikasjon i IDS
TEK_ids.specifications.append(oppbevaring_av_torrmat_spec)

In [5]:
# Oppbevaring av mat spesifikasjon
oppbevaring_av_kjolevarer_spec = ids.Specification(
    name="§12-10 (2) (2) Plass til oppbevaring av mat (Kjølevarer)",
    description= "Boenheten skal ha tilstrekkelig og egnet plass for oppbevaring av kjølevarer",
    instructions="Produksjonsunderlag (BIM) må vise avsatt plass til oppbevaring av kjølevarer med riktig object type.",
    minOccurs=1,
    ifcVersion="IFC4")

oppbevaring_av_kjolevarer_spec.applicability.append(ids.Entity(name = "IfcElectricAppliance"))
#oppbevaring_av_kjolevarer_spec.applicability.append(ids.Attribute(
#    name="ObjectType",value="STORAGE_REFRIGERATED_GOODS"))
#oppbevaring_av_kjolevarer_spec.applicability.append(ids.Attribute(name="ObjectType",value="REFRIDGERATOR"))
oppbevaring_av_kjolevarer_spec.applicability.append(ids.Attribute(name="PredefinedType",value="FRIDGE_FREEZER"))

name = ids.Attribute(
    name = "Name"
)
oppbevaring_av_kjolevarer_spec.requirements.append(name)

longName = ids.Attribute(
    name = "LongName"
)
oppbevaring_av_kjolevarer_spec.requirements.append(longName)

# Legge til spesifikasjon i IDS
TEK_ids.specifications.append(oppbevaring_av_kjolevarer_spec)

In [6]:
# Skriv til fil
TEK_ids.to_xml("../ids/§12_10_TEK_IfcFuniture_and_IfcElectricAppliance_Error_LongName.ids")

True

In [10]:
file = ifcopenshell.open("../ifc/tek-bim.ifc")
tekids = ids.open("../ids/§12_10_TEK_IfcFuniture_and_IfcElectricAppliance_Error_LongName.ids")
tekids.validate(file)

In [11]:
reporter.Console(tekids).report()

TEK og BIM
[FAIL] (0/1) *§12-10 (1) Plass til oppbevaring av klær
    Applies to:
        All IfcSpatialZone data
        Data where the ObjectType is STORAGE_OF_CLOTHES
    Requirements:
        The Name shall be provided
        The LongName shall be provided
            The attribute value None is empty - #2911=IfcSpatialZone('0T9InqK9T2BvhcCuJN9byU',$,'Oppbevaring av klær',$,'STORAGE_OF_CLOTHES',#3745,#2917,$,.USERDEFINED.)
[FAIL] (0/1) *§12-10 (2) (1) Plass til oppbevaring av mat (Tørrmat)
    Applies to:
        All IfcSpatialZone data
        Data where the ObjectType is STORAGE_DRY_FOOD
    Requirements:
        The Name shall be provided
        The LongName shall be provided
            The attribute value None is empty - #2930=IfcSpatialZone('2DQ02zAnL4deegroDOcto7',$,'Oppbevaring av tørrmat',$,'STORAGE_DRY_FOOD',#3750,#2936,$,.USERDEFINED.)
[FAIL] (0/1) *§12-10 (2) (2) Plass til oppbevaring av mat (Kjølevarer)
    Applies to:
        All IfcElectricAppliance data
        Da